In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

In [2]:
torch.cuda.is_available()

True

In [3]:
model = model.cuda()

In [4]:
TOKEN = '5213574179:AAFRp9HUe7JvbJtUIWfGsJAMQH25uKvWzm8'

In [5]:
import telebot

In [6]:
bot = telebot.TeleBot(TOKEN)

In [7]:
chat_history_ids = {}

In [8]:
# start dialog
@bot.message_handler(commands=['start', 'help'])
def send_welcome(msg):
    if msg.chat.id in chat_history_ids:
        chat_history_ids.pop(msg.chat.id)
    bot.reply_to(msg, "Hi, this is dialog bot based on DIALOGPT-medium. Try to start conversation in english.")

In [49]:
@bot.message_handler(commands=['restart'])
def send_welcome(msg):
    if msg.chat.id in chat_history_ids:
        chat_history_ids.pop(msg.chat.id)
    bot.reply_to(msg, "Hi, this is dialog bot based on DIALOGPT-medium. Try to start conversation in english.")

In [9]:
@bot.message_handler(func=lambda msg: True)
def echo_all(msg):
    usr_id = msg.chat.id
    if usr_id in chat_history_ids:
        usr_msg = " " + msg.text
    else:
        usr_msg = msg.text
    
    print("User: " + str(usr_id) + " msg: " + msg.text)
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(usr_msg + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids[usr_id], new_user_input_ids], dim=-1) if usr_id in chat_history_ids else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids[usr_id] = model.generate(bot_input_ids.cuda(), max_length=1000, pad_token_id=tokenizer.eos_token_id).cpu()

    # pretty print last ouput tokens from bot
    answer = tokenizer.decode(chat_history_ids[usr_id][:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    chat_history_ids[usr_id] = chat_history_ids[usr_id][:, max(bot_input_ids.shape[-1]-50, 0):]
    if answer == '':
        chat_history_ids.pop(usr_id)
    print("DialoGPT: {}".format(answer))

    bot.reply_to(msg, answer)

In [ ]:
bot.polling()

User: 394194431 msg: Hi
DialoGPT: Hi! :D
User: 394194431 msg: How are you?
DialoGPT: I'm good, how are you?
User: 394194431 msg: Hi
DialoGPT: Hi! :D
User: 120342957 msg: Hi
DialoGPT: Hi! :D
User: 120342957 msg: What are you doing
DialoGPT: I'm going to sleep.
User: 120342957 msg: 😴 good night
DialoGPT: Night! :D
User: 394194431 msg: Do you know about virtual machine snapshot?
DialoGPT: I don't know about that, but I know about virtual machine snapshots.
User: 394194431 msg: Can you help me?
DialoGPT: I can help you.
User: 394194431 msg: Tell me about volume storage
DialoGPT: I can help you with that.
User: 394194431 msg: 😃
DialoGPT: I can help you with that too.
User: 394194431 msg: Nice to hear that
DialoGPT: I can help you with that too.
User: 394194431 msg: How is the weather
DialoGPT: It's nice.
User: 185471179 msg: Hello fucker
DialoGPT: Hey, I'm not the one who made the post.
User: 400664727 msg: ahahaha
DialoGPT: I'm not sure if you're serious or not.
User: 361984038 msg: how ma